# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tilichiki,60.4667,166.1000,12.09,86,99,8.39,RU,1635033059
1,1,punta arenas,-53.1500,-70.9167,35.71,87,90,27.63,CL,1635033061
2,2,praya,14.9215,-23.5087,77.54,78,0,12.66,CV,1635033064
3,3,antofagasta,-23.6500,-70.4000,60.08,77,6,7.76,CL,1635033066
4,4,severodvinsk,64.5635,39.8302,24.91,83,100,4.94,RU,1635033068


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
164,164,mecca,21.4267,39.8261,79.48,59,0,0.69,SA,1635033458
196,196,gwadar,25.1216,62.3254,77.79,67,0,3.04,PK,1635033546
272,272,cayenne,4.9333,-52.3333,78.10,89,0,0.00,GF,1635033609
274,274,koungou,-12.7336,45.2042,78.51,83,0,2.30,YT,1635033748
357,357,dondo,-19.6094,34.7431,71.44,87,0,3.29,MZ,1635033962
373,373,sur,22.5667,59.5289,79.63,68,0,9.37,OM,1635033999
514,514,beira,-19.8436,34.8389,74.01,85,0,7.52,MZ,1635034346


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
164,mecca,SA,21.4267,39.8261,
196,gwadar,PK,25.1216,62.3254,
272,cayenne,GF,4.9333,-52.3333,
274,koungou,YT,-12.7336,45.2042,
357,dondo,MZ,-19.6094,34.7431,
373,sur,OM,22.5667,59.5289,
514,beira,MZ,-19.8436,34.8389,


In [7]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df.tail()

{'html_attributions': [], 'next_page_token': 'Aap_uEDtKE-bOgg4WDeCONMZXUG9ufHSc_thFCnjkHw8Gk3yILP5SJH2wkJ8cU-WbITirTwaQxM8BvGkMzK3xeOEV71nfimSLKtTCp-tKQO8zFPMqmK9xb1tmOIBlPSaQo5_BKdgw2xdhQAJFbYvMkxBhmLZG3t295RyvFK7kEMJwC11rSCJroayb7b9-GIh5nSq4dCKhRjHUX-_QPqSGvzgo5rOOIgjORiLsTXmpI4xhQNdrCBB4lvTtGLkcSUS1saLQvCjTZXjMA9mpwvG-5pW6b7_As_pL24A1_x1MaTrAUmDBoiH0kzXb2ip3u6302Nz3uB3sUfE0K39IrVEXvlx9nhlyRuvnQtg3nVEiB-wIFW2KPHpTyKjjfI6tjxDAYAk4UFTbmkAccej1AkTQorYldtj6OuHjWvoi6sgD9-BVsn_HSZFTbX8SjHH', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 21.418034, 'lng': 39.82556899999999}, 'viewport': {'northeast': {'lat': 21.4196736802915, 'lng': 39.82804040000001}, 'southwest': {'lat': 21.4169757197085, 'lng': 39.8247452}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Makkah Clock Royal Tower, A

{'html_attributions': [], 'next_page_token': 'Aap_uEA4DgGJCdeb3wPuhbHK6x84EdjFJSgIn3KKXPqC4HT1JnTxsVNfgNdeESpGG6XZORncFSKrd7dRLWPzk7FVnkrJtdRJo4ogk7JgfY5gKGynjlfJRgATWoH6T0NJeGs_v-zLldbvraPKCHjtL19S5aW8kYbK8bkZSrTkZkobRztQqoGgUx_q9LVLo9kkghkFewWMjV4R2xn5xDSw81tO2eGQzjbEl4uzMJqeUs3f91mm17vztfaeIcYJ3myO_MnvxmEi4Y1n9rweJz4TImLXlee-8DVmrgofZc3md97tOtkOaJvvmlmkMRPrlQRV26RanbA5v1jgBfh9htIjcvANb-oo-nOefIHsP2H2n6eDiMr1xTG3yLcNZ11SG-qG1repPCmCe5TalFoEDiuiKQNynw8ZvG0ph_4YuJyn1f1SYhq3UdO3owOBPGgh', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 25.126028, 'lng': 62.31964540000001}, 'viewport': {'northeast': {'lat': 25.1274082302915, 'lng': 62.3210022302915}, 'southwest': {'lat': 25.1247102697085, 'lng': 62.3183042697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Sadaf Resort', 'openin

{'html_attributions': [], 'next_page_token': 'Aap_uEAD7P2FaI1XeJAnah_Biy_J6rS8r8XfUS13LsKbB6h6XvuhEte9cmxO3o-uSmbb97DytDiIn_tXOsnrL0P2UsUzPidLalJKzPcdgsMpPTbf0lwFc9dsxkRYhqBk_wij-3hTBw2Cjd8mPLhPfjDGbM8VK2jS_lr4JbkAJ83sHG07D32oKC2ChR3Yj2ugVlQRZQNk2HhqClUdD8tY9pODU6FfdrA70fg6L2wpADZmvfQdR08i46IAmhd4JZuX8-vaP7xrm5ZLqD_vP33WvsEer__5Kh9otaohLrcoMDn94iavnslv6EX4rQL6_Di14LPxus3AQoj5r7Y3_0dYZsXTexuBLbxHhUr6e4rxtSGF9GfsB7cazUTRf67rJgSFWgPyfBWRlysYRYaxeU_jn7PlOxTVBAF2O_EUI3HOvwMV3kN44NoI8_nrTQbb', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 4.9387955, 'lng': -52.3314083}, 'viewport': {'northeast': {'lat': 4.940208530291502, 'lng': -52.3300759697085}, 'southwest': {'lat': 4.937510569708498, 'lng': -52.33277393029149}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Hôtel Le Dronmi', 'open

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -12.7348533, 'lng': 45.21271369999999}, 'viewport': {'northeast': {'lat': -12.73352671970849, 'lng': 45.2140853802915}, 'southwest': {'lat': -12.7362246802915, 'lng': 45.2113874197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Villa Netibor', 'place_id': 'ChIJDYvBKrJsCiIRBkO6rKoDVtw', 'rating': 5, 'reference': 'ChIJDYvBKrJsCiIRBkO6rKoDVtw', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'user_ratings_total': 1, 'vicinity': 'POINTE DE KOUNGOU, 3 Le Belvédère, Koungou'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -12.7313888, 'lng': 45.194546}, 'viewport': {'northeast': {'lat': -12.7300398197085, 'lng': 45.1958949802915}, 'southwest': {'lat': -12.7

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -19.6115404, 'lng': 34.7409293}, 'viewport': {'northeast': {'lat': -19.6099953197085, 'lng': 34.7427975302915}, 'southwest': {'lat': -19.6126932802915, 'lng': 34.7400995697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Arco Iris', 'photos': [{'height': 2592, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114887855777691022818">RANDINHO Manuel</a>'], 'photo_reference': 'Aap_uEAI-uAiTCeJJvdAROrbbduialO-ZJlpBTxf5XM1sZrOrhV4BQjMn_fK0GkcrEIp2wcMvycniMYl_xadlvoo2vmK9r_GDqL0ldyOX6bzoqR_lUrMvNMNubyLEqjC0OOme8oNGKweXTqhIlSjr3RnndYknbWCv2T8rGK388z5JP0Zgo6s', 'width': 1456}], 'place_id': 'ChIJAQAAALBjKh8RPQbGGXgbCHw', 'rating': 4.7, 'reference': 'ChIJAQAAALBjKh8RPQbGGXgbCHw', 'scope': 'GOOGL

{'html_attributions': [], 'next_page_token': 'Aap_uEDquas17X-6ci7dxBpBPfPzMa3DZGiDy-SY3Qoevamp5DR7SnusNTYQ8kCD7fH3waRPj_EHr1nQFWxn_XcVW_v0JPDmy_kYH3A9Vwz4cG9y-jtaBrUemoKE4GDrdEvIwEVpyVgd8TBQKztUjWDtuwvG_wVbPmenef7vdGQroFHxBsyKt7aFm_iXeHDkudDqJLhSQEAgkg5sroAB9N6wGYR1VokQcpVl2Nx8exNyAHp4OaQPqD_FYDBOdLlh0s0OYRrY2wHkqP6gNaBWpHoDZ8oQH4sVEtWE3BB9Izq7otzJR7YHD5gd97xh0MxlgIZwfdFUpIJx3vWcfI7F0aUCuWrmH-sIN2iPPinMC4stxKXgOi3CuUa1EoDG4-4wKQs38shJBx0SZiDPOKIpAIPfvA2WphgtUf8uE79k-dWVdI0s82gJqxDKAldi', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 22.5664354, 'lng': 59.4958585}, 'viewport': {'northeast': {'lat': 22.5678508302915, 'lng': 59.49709478029151}, 'southwest': {'lat': 22.5651528697085, 'lng': 59.4943968197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Sur Plaza Hotel', 'photos':

{'html_attributions': [], 'next_page_token': 'Aap_uEC0U_cmc2KgRKKS178SxuURShcsF4ROcmcje6tm7lzzPCXvqjvuBUakP7n9La10GRcD4lNa9QUigK4iwhuivTc2R1-0u8oksjUA-Ufh1girigcRkqJ9ASW5jnrY4HwysyxX-0qRfbZk8tV36obl7qi9mkkoSmMdQIUUPBO905HygD4hB7HRus1ljQFORP0GcrjGzPWVG3Io5hfp3JBpNNLZeea3mohLytWfZWpky-uv43KWTohn9liUSLbeRZGaqpyNw9-317teKb_ELNl8ZSPxtFgq9HZ0bgJRNK3k64hmHtaF_zSJLXXUVjZWK_41nFWOFHncn7u7_nEiJkVJwy1lJbum6sZT7g_5rTxlatOx5hLn2QCUQqXNiIB5xNOWIlLkhwJOvrAloplgYY2XnWFPt0abVyuXrdvPnRfZNI0QV__j4n3RqJyq', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -19.8282649, 'lng': 34.8325481}, 'viewport': {'northeast': {'lat': -19.8268626697085, 'lng': 34.8337076802915}, 'southwest': {'lat': -19.8295606302915, 'lng': 34.8310097197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Beira Terrace Hotel', 'ph

,City,Country,Lat,Lng,Hotel Name
272,cayenne,GF,4.9333,-52.3333,Hôtel Le Dronmi
274,koungou,YT,-12.7336,45.2042,Villa Netibor
357,dondo,MZ,-19.6094,34.7431,Arco Iris
373,sur,OM,22.5667,59.5289,Sur Plaza Hotel
514,beira,MZ,-19.8436,34.8389,Beira Terrace Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))